# Assignment Part 3: Segmentation of Human Blood Cells using Convolutional Neural Networks
*By David Holmberg*

Welcome to Assignment part 3! In this assignment you will segment blood cells using some both scratch-built models and compare to pre-defined downloaded models.

Hand-in: this notebook or a pdf of it.

For this lab, we use the [Human White Blood Cell images](https://github.com/zxaoyou/segmentation_WBC) from [Jiangxi Tecom Science Corporation, China](http://en.tecom-cn.com/).

The dataset contains three hundred 120x120 RGB images with one blood cell per image, and corresponding segmentation masks. The segmentation mask was manually sketched by domain experts, with the background, cytoplasms and nuclei pixels labelled as 0, 1 and 2 respectively.

<img src="Illustrations/Dataset_1.png" title="Blood cells dataset" align="left"/>




These images and masks are in the **LabData/bloodcells_seg/** folder:
```
└── LabData
    └── bloodcells_seg
        ├── masks
        │   ├── all
        └── images
            ├── all
```

We want to use convolutional neural networks to do pixel-wise classification of these blood cells images into background / cytoplasm / nuclei.

This assignment is based on and borrows *heavily* from a lab written by Christophe Avenel at NBIS, and his code is available here: https://github.com/NBISweden/workshop-neural-nets-and-deep-learning/tree/master/session_convolutionalNeuralNetworks/Labs

So please remember that he is the original author of this assignment.

In [ ]:
# First we import useful modules like numpy and Keras layers

import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
# cnn_helper contains some useful functions for this lab
import cnn_helper
from IPython.display import IFrame
import tensorflow as tf
from tensorflow import keras
from keras import layers
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import os
os.environ["SM_FRAMEWORK"]="tf.keras"
import segmentation_models as sm


We will load both the images and the masks as generators, and combine both generators into a zip of generators.

In [ ]:
IMG_SIZE = 128
NUM_CLASSES = 3 # Background, cytoplasm, nuclei
N_CHANNELS = 3  # R,G,B
BATCH_SIZE = 8  # You can tune this and see which batch size runs faster / gives the best accuracy.


train_steps = 240/BATCH_SIZE
val_steps = 64/BATCH_SIZE


seed = 909 # (IMPORTANT) to transform image and corresponding mask with same augmentation parameter.
image_datagen = ImageDataGenerator(
    validation_split=0.2,
    rescale=1./255 #We need to rescale png images from 0-255 integers to 0.0-1.0 floats.
)
mask_datagen = ImageDataGenerator(
    validation_split=0.2
)

# Load image and mask generators for training:
train_image_generator = image_datagen.flow_from_directory(
    directory='LabData/bloodcells_seg/images/',
    batch_size=BATCH_SIZE,
    target_size=(IMG_SIZE,IMG_SIZE),
    class_mode=None, 
    seed=seed,
    subset='training',
    color_mode='rgb',
    shuffle=False)
# Note: the class_mode is used when the class is extracted from the directory
# structure. Here the class will be defined by the mask from the generator below, 
# so we add class_mode: None

train_mask_generator = mask_datagen.flow_from_directory(
    directory='LabData/bloodcells_seg/masks/',
    batch_size=BATCH_SIZE,
    target_size=(IMG_SIZE,IMG_SIZE),
    class_mode=None,
    seed=seed,
    subset='training',
    color_mode='grayscale',
    shuffle=False)

train_generator = zip(train_image_generator, train_mask_generator)

# Load image and mask  generators for validation:
val_image_generator = image_datagen.flow_from_directory(
    directory='LabData/bloodcells_seg/images/',
    batch_size=BATCH_SIZE,
    target_size=(IMG_SIZE,IMG_SIZE),
    class_mode=None,
    seed=seed,
    subset='validation',
    color_mode='rgb',
    shuffle=False)

val_mask_generator = mask_datagen.flow_from_directory(
    directory='LabData/bloodcells_seg/masks/',
    batch_size=BATCH_SIZE,
    target_size=(IMG_SIZE,IMG_SIZE),
    class_mode=None,
    seed=seed,
    subset='validation',
    color_mode='grayscale',
    shuffle=False)

val_generator = zip(val_image_generator, val_mask_generator)

In the following cell we're defining the U-NET model. This model will break down the image into it's base components and reconstruct it into masks. Then we can use it to create binary masks for all cells for future segmentation.


<img src="Illustrations/unet.png" title="Unet model"/>

In [ ]:
inputs = layers.Input((IMG_SIZE, IMG_SIZE, N_CHANNELS))

c1 = layers.Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') (inputs)
c1 = layers.Dropout(0.1) (c1)
c1 = layers.Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') (c1)
p1 = layers.MaxPooling2D((2, 2)) (c1)

c2 = layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') (p1)
c2 = layers.Dropout(0.1) (c2)
c2 = layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') (c2)
p2 = layers.MaxPooling2D((2, 2)) (c2)

c3 = layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') (p2)
c3 = layers.Dropout(0.2) (c3)
c3 = layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') (c3)
p3 = layers.MaxPooling2D((2, 2)) (c3)

c4 = layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') (p3)
c4 = layers.Dropout(0.2) (c4)
c4 = layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') (c4)
p4 = layers.MaxPooling2D(pool_size=(2, 2)) (c4)

c5 = layers.Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') (p4)
c5 = layers.Dropout(0.3) (c5)
c5 = layers.Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') (c5)

u6 = layers.Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same') (c5)
u6 = layers.concatenate([u6, c4])
c6 = layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') (u6)
c6 = layers.Dropout(0.2) (c6)
c6 = layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') (c6)

u7 = layers.Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same') (c6)
u7 = layers.concatenate([u7, c3])
c7 = layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') (u7)
c7 = layers.Dropout(0.2) (c7)
c7 = layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') (c7)

u8 = layers.Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same') (c7)
u8 = layers.concatenate([u8, c2])
c8 = layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') (u8)
c8 = layers.Dropout(0.1) (c8)
c8 = layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') (c8)

u9 = layers.Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same') (c8)
u9 = layers.concatenate([u9, c1], axis=3)
c9 = layers.Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') (u9)
c9 = layers.Dropout(0.1) (c9)
c9 = layers.Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') (c9)

outputs = layers.Conv2D(NUM_CLASSES, (1, 1), activation='softmax') (c9)

While **RELU** is the goto-function, there are other options. Typically you'd want to chose anything from the **RELU** family of activations functions (relu, selu, relu6, elu).

You can visualize and compare activation functions with this online tool from [Justin Emery](https://tech.courses/author/justinjustinemery-co-uk/):

In [ ]:
IFrame('https://polarisation.github.io/tfjs-activation-functions/', width=860, height=470)

We will use the Keras SparseCategoricalCrossentropy loss function (see https://www.tensorflow.org/api_docs/python/tf/keras/losses/SparseCategoricalCrossentropy).

because our ground truth is represented by integers on a mask (0 for background, 1 for cytoplasm, 2 for nuclei pixels) this becomes a good choice.

Please define, summarize, and compile it in the following three cells:

**tip:** you'll want to define the loss specifically, i.e use tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False) and not 'SparseCategoricalCrossentropy' as before.

**tip:** you can define the learning rate specifically in Adam() inside the parenthesis. I suggest you start with 0.0001.

In [ ]:
## Define model here:
model = keras.Model(inputs=[inputs], outputs=[outputs])

In [ ]:
#Summarize model here:

In [ ]:
#Compile model here using optimizer=Adam and loss=tf.keras.losses.SparseCategoricalCrossentropy (Hint; Google it!). You use the compilation the same way as in previous assignments:


Now you will train the network.

Look back at how you trained your model in the previous Assignment.

You can track the progress of your training using the callback function we've defined for you in the cell below. you can do this by adding ```callbacks=[plot_callback]``` at the end of your model.fit_generator function. Since this is the first time you're using unsupervised, I've provided the fit_generator function for you.

In [ ]:
#plot callback:
epochs = ???

plot_callback = cnn_helper.PlottingKerasCallback(
    test_batch=next(val_generator),
    num_plot=3 # how many validation examples to plot at each epoch (maximum = BATCH_SIZE)
)

model.fit_generator(
    generator=train_generator,
    steps_per_epoch= train_steps,
    validation_data= val_generator,
    validation_steps= val_steps,
    epochs= epochs,
    callbacks=[plot_callback]
)

You can use the following cell to batch plot som validation. 

In [ ]:
cnn_helper.plot_prediction(model, next(val_generator), BATCH_SIZE)


Try improving on your model by changing the learning rate, epochs, activation function or any other parameters you want. I suggest you stick to the training parameters to begin with, as they are easier to tune.


# Other models

To simplify the construction of your networks, you will now use the `segmentation_model` python library ( it should be installed already, but if you get an error, install using `pip install segmentation_model`).
 
You can read more about the `segmentation_model` library at https://github.com/qubvel/segmentation_models.

## List of models:

| Unet | Linknet |
| --- | --- |
| <img width="500px" src="https://raw.githubusercontent.com/qubvel/segmentation_models/master/images/unet.png" title="UNet"/> | <img width="500px" src="https://github.com/qubvel/segmentation_models/raw/master/images/linknet.png" title="Linknet"/> |

| PSPNet | FPN |
| --- | --- |
| <img width="500px" src="https://github.com/qubvel/segmentation_models/raw/master/images/pspnet.png" title="UNet"/> | <img width="500px" src="https://github.com/qubvel/segmentation_models/raw/master/images/fpn.png" title="Linknet"/> |

## List of backbones:

| Type | Names |
| --- | --- |
| VGG | `vgg16` `vgg19` |
| ResNet | `resnet18` `resnet34` `resnet50` `resnet101` `resnet152` |
| SE-ResNet | `seresnet18` `seresnet34` `seresnet50` `seresnet101` `seresnet152` |
| ResNeXt | `resnext50` `resnext101` |
| SE-ResNeXt | `seresnext50` `seresnext101` |
| SENet154 | `senet154` |
| DenseNet | `densenet121` `densenet169` `densenet201` |
| Inception | `inceptionv3` `inceptionresnetv2` |
| MobileNet | `mobilenet` `mobilenetv2` |
| EfficientNet | `efficientnetb0` `efficientnetb1` `efficientnetb2` `efficientnetb3` `efficientnetb4` `efficientnetb5` `efficientnetb6` `efficientnetb7` |

Now try repeating the segmentation using a combination of either UNET or LinkNet with one of the backbones. To save your from the headache of trying to use an unfamiliar python package we've provided an example of using UNET with the VGG16 back-end you used in the previous assignment.

Try playing around a bit. Unet and Linknet are used for the type of tasks we are performing here, and as such I recommend you to use them here as well.

PN (Feature Pyramid Networks) are mainly used for object detection, and PSPNet (Pyramid Scene Parsing Networks) are not covered here, but you can read about them on https://towardsdatascience.com/review-pspnet-winner-in-ilsvrc-2016-semantic-segmentation-scene-parsing-e089e5df177d.

In [ ]:
# Let's load the Unet model, using the VGG16 backbone
model = sm.Unet(
    'vgg16',
    encoder_weights=None,
    classes=NUM_CLASSES,
    activation='softmax',
    input_shape=(IMG_SIZE,IMG_SIZE,3)
)


In [ ]:
#Print the model summary:



You will now define, compile, and train the new model you selected from the module with the same parameters as your best run with your self-built model.



In [ ]:
#compile here, same way as before. Same optimizer, same loss function:




In [ ]:
#train model here. Hint; you can use the same fit_generator function as before. Just make sure to rename the variables correctly!:


In [ ]:
#Print prediction the same way as before:


## End of Assignment part 3:

And this is how image segmentation is done. I hope you enjoyed this brief introduction, and encourage you to play around and test multiple models, both predefined and scratch-built.